In [1]:
import pandas as pd
import numpy as np 
import os
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, make_scorer, roc_auc_score, mean_squared_error,mean_absolute_error
from sklearn.cluster import KMeans
from scipy import stats
from sklearn.model_selection import GridSearchCV
from prophet.diagnostics import cross_validation
import joblib
# import optuna
# import xgboost as xgb
import catboost
import lightgbm as lgb
from skopt import BayesSearchCV
from prophet import Prophet
from prophet.diagnostics import performance_metrics
import xgboost as xgb

In [2]:
def y_changer(data):
    data = data.rename(columns = {"y": "평균기온", "월":"months"})
    return data

In [3]:
data = pd.read_csv('./prep_data.csv')
data.ds = pd.to_datetime(data.ds)
data = data.rename(columns = {"평균기온":"y"})
data['days'] = data.ds.dt.day
data['years'] = data.ds.dt.year
summit_form = pd.read_csv('./sample_submission.csv')
regressors = [x for x in data.columns if x != "y" and x != "ds"]
data = y_changer(data)

In [4]:
data.columns

Index(['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율',
       '평균기온', 'months', 'ds', 'days', 'years'],
      dtype='object')

In [5]:
X = data.drop(["평균기온",'일교차', '일조합','ds'], axis = 1)
y = data.평균기온

In [33]:
y_new = y[365:]

In [34]:
X_new = X[:-365]

In [12]:
new

22646

In [13]:
len(y_new)

22646

In [40]:
X_train, X_test = X_new[:-365], X_new[-365:]
y_train, y_test = y_new[:-365], y_new[-365:]

In [41]:
X_test

,최고기온,최저기온,강수량,평균습도,평균풍속,일사합,일조율,months,days,years
22281,1.6,-9.8,0.61,64.0,2.0,9.54,67.7,1.0,1,2021
22282,-1.4,-8.4,0.18,38.5,2.6,11.34,93.8,1.0,2,2021
22283,-2.0,-9.1,0.09,45.0,2.0,8.80,56.7,1.0,3,2021
22284,0.3,-8.4,0.00,51.4,1.7,7.44,47.4,1.0,4,2021
22285,-2.1,-9.9,0.00,52.8,2.9,10.46,88.7,1.0,5,2021
...,...,...,...,...,...,...,...,...,...,...
22641,-3.9,-12.9,0.00,60.9,1.7,6.09,39.6,12.0,27,2021
22642,-0.9,-8.5,0.66,73.8,2.2,4.32,17.7,12.0,28,2021
22643,5.9,-3.8,0.20,72.9,2.6,4.64,18.8,12.0,29,2021
22644,0.2,-6.8,0.00,48.5,3.3,9.14,76.0,12.0,30,2021


In [45]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from skopt import BayesSearchCV



# 하이퍼파라미터 탐색 범위 정의
param_space = {
    'learning_rate': (0.01, 1.00, 'uniform'),
    'max_depth': (3, 12),
    'n_estimators': (100, 1000),
    'subsample': (0.10, 1.00, 'uniform'),
    'colsample_bytree': (0.10, 1.00, 'uniform'),
    'gamma': (0, 1.0, 'uniform'),
}

# XGBoost 모델 초기화
model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

# BayesSearchCV를 사용하여 베이지안 최적화 수행
opt = BayesSearchCV(
    model,
    param_space,
    scoring='neg_mean_absolute_error',
    n_iter=50,  # 반복 횟수
    cv=5,  # 교차 검증 폴드 수
    n_jobs=1,  # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
)

# 최적의 하이퍼파라미터로 모델 학습
opt.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", opt.best_params_)

# 최적의 모델 평가 (여기서는 교차 검증 점수 출력)
print("Cross-Validation Score:", -opt.best_score_)


Best Parameters: OrderedDict([('colsample_bytree', 0.8780741118210806), ('gamma', 0), ('learning_rate', 0.12473347174963892), ('max_depth', 8), ('n_estimators', 456), ('subsample', 0.6080385860401739)])
Cross-Validation Score: 2.758964698078302


In [47]:
opt.best_estimators

AttributeError: 'BayesSearchCV' object has no attribute 'best_estimators'

In [48]:
best_model = opt.best_estimator_

In [58]:
pred = best_model.predict(X_test)

mean_absolute_error(y_test, pred)

mae = mean_absolute_error(y_test, pred)

In [118]:
def select_label(data, labels, label):

    # X = data.drop(["평균기온",'일교차', '일조합','ds', label], axis = 1)
    X = data[labels].drop([label], axis = 1)
    y = data[label]
    
    return X, y 

def train_test_data(X,y):
    
    X_train, X_test = X[:-365], X[-365:]
    y_train, y_test = y[:-365], y[-365:]
    
    return X_train, X_test, y_train, y_test

def make_model(X_train, y_train):
    
    # 하이퍼파라미터 탐색 범위 정의
    param_space = {
        'learning_rate': (0.01, 1.00, 'uniform'),
        'max_depth': (3, 12),
        'n_estimators': (100, 1000),
        'subsample': (0.10, 1.00, 'uniform'),
        'colsample_bytree': (0.10, 1.00, 'uniform'),
        'gamma': (0, 1.0, 'uniform'),
    }

    # XGBoost 모델 초기화
    model = xgb.XGBRegressor(objective="reg:squarederror", n_jobs = -1, random_state=42)

    # BayesSearchCV를 사용하여 베이지안 최적화 수행
    opt = BayesSearchCV(
        model,
        param_space,
        scoring='neg_mean_absolute_error',
        n_iter=50,  # 반복 횟수
        cv=5,  # 교차 검증 폴드 수
        # n_jobs=1,  # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
    )

    # 최적의 하이퍼파라미터로 모델 학습
    opt.fit(X_train, y_train)

    # 최적의 하이퍼파라미터 출력
    print("Best Parameters:", opt.best_params_)

    # 최적의 모델 평가 (여기서는 교차 검증 점수 출력)
    print("Cross-Validation Score:", -opt.best_score_)
    
    return opt
    

def socoring(opt, y_test, X_test):
    
    pred = opt.best_estimator_.predict(X_test)

    mean_absolute_error(y_test, pred)

    mae = mean_absolute_error(y_test, pred)
    print(mae)
    
    return mae

def progress(data, labels):
    
    
    data = data
    
    labels = labels
    
    for label in labels:
    
        X, y = select_label(data, labels, label)

        X_train,X_test, y_train, y_test = train_test_data(X,y)

        opt = make_model(X_train,y_train)

        label = label + "_model"
        
        print(label)

        globals()[label] = opt

        mae = socoring(opt, y_test, X_test)

        return print(mae)

def modeling(params, X_train, y_train):
    
    model = xgb.XGBRegressor(objective="reg:squarederror", **params, n_jobs = -1, random_state=42)
    
    model.fit(X_train, y_train)
    
    return model

def model_predict(data, label, params):
    
    data = data
    
    label = label
    
    X, y = select_label(data, label)
    
    X_train,X_test, y_train, y_test = train_test_data(X,y)
    
    model = modeling(params, X_train, y_train)
    
    model_pred = model.predict(X_test)
    
    return model_pred
    

In [113]:
avg_tmp = dict([('colsample_bytree', 0.7489488665614384), ('gamma', 0), ('learning_rate', 0.2303112080128232), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 1.0)])
high_tmp = dict([('colsample_bytree', 0.785843038567719), ('gamma', 1), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 0.36837214707260824)])
low_tmp = dict([('colsample_bytree', 0.9307064570326005), ('gamma', 0), ('learning_rate', 0.01), ('max_depth', 5), ('n_estimators', 1000), ('subsample', 1.0)])
avg_hum = dict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.011022102502535366), ('max_depth', 4), ('n_estimators', 1000), ('subsample', 0.11499728444446672)])

In [114]:
avg_tmp

{'colsample_bytree': 0.7489488665614384,
 'gamma': 0,
 'learning_rate': 0.2303112080128232,
 'max_depth': 3,
 'n_estimators': 1000,
 'subsample': 1.0}

In [102]:
labels = ["평균기온", "최고기온", "최저기온", "평균습도"]
for label in labels :
    progress(data, labels)

Best Parameters: OrderedDict([('colsample_bytree', 0.7489488665614384), ('gamma', 0), ('learning_rate', 0.2303112080128232), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 1.0)])
Cross-Validation Score: 0.39732766430568645
0.3690247294970164
0.3690247294970164
Best Parameters: OrderedDict([('colsample_bytree', 0.785843038567719), ('gamma', 1), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 0.36837214707260824)])
Cross-Validation Score: 0.39090319278662566
0.3763858020795535
0.3763858020795535
Best Parameters: OrderedDict([('colsample_bytree', 0.9307064570326005), ('gamma', 0), ('learning_rate', 0.01), ('max_depth', 5), ('n_estimators', 1000), ('subsample', 1.0)])
Cross-Validation Score: 0.3902358750681939
0.37210015173235994
0.37210015173235994
Best Parameters: OrderedDict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.011022102502535366), ('max_depth', 4), ('n_estimators', 1000), ('subsample', 0.11499728444446672)])
Cross-Valid

In [104]:
print(최고기온_model.best_params_)
print(최저기온_model.best_params_)
print(평균습도_model.best_params_)
print(평균기온_model.best_params_)

OrderedDict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.01), ('max_depth', 3), ('n_estimators', 1000), ('subsample', 1.0)])
OrderedDict([('colsample_bytree', 1.0), ('gamma', 1), ('learning_rate', 0.219519027392961), ('max_depth', 8), ('n_estimators', 1000), ('subsample', 0.5209965334687984)])
OrderedDict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.01), ('max_depth', 5), ('n_estimators', 978), ('subsample', 0.6873473201351862)])
OrderedDict([('colsample_bytree', 1.0), ('gamma', 0), ('learning_rate', 0.011022102502535366), ('max_depth', 4), ('n_estimators', 1000), ('subsample', 0.11499728444446672)])


In [125]:
label_dic = {"최고기온" : high_tmp, "최저기온" : low_tmp, "평균습도" : avg_hum}

In [126]:

X, y = select_label(data, labels, "평균기온")

X_train,X_test, y_train, y_test = train_test_data(X,y)

base_m = modeling(avg_tmp, X_train, y_train)\

for k,v in label_dic.items() :
    
    X1, y1 = select_label(data, labels, k)
    
    X_train1,X_test1, y_train1, y_test1 = train_test_data(X1,y1)
    
    print(v)
    
    m = modeling(v, X_train1, y_train1)
    
    X_test[k] = m.predict(X_test1)
    

# y_pred = base_m.predict(X_test)

# mae = mean_absolute_error(y_test, pred)

# print(mae)

{'colsample_bytree': 0.785843038567719, 'gamma': 1, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 1000, 'subsample': 0.36837214707260824}
{'colsample_bytree': 0.9307064570326005, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 1000, 'subsample': 1.0}
{'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.011022102502535366, 'max_depth': 4, 'n_estimators': 1000, 'subsample': 0.11499728444446672}


In [127]:
y_pred = base_m.predict(X_test)

mae = mean_absolute_error(y_test, pred)

print(mae)

2.8153716568097678


In [128]:
X_test

,최고기온,최저기온,평균습도
22646,-0.155919,-8.238092,60.601475
22647,3.271398,-5.182496,56.391415
22648,2.114505,-5.983513,57.746731
22649,2.158552,-6.311744,54.248131
22650,1.312312,-6.560971,55.886009
...,...,...,...
23006,1.562991,-6.547873,54.769093
23007,1.144488,-6.849960,56.458641
23008,1.172052,-6.911513,56.207546
23009,2.602852,-5.330601,57.111187


In [ ]:

X, y = select_label(data, labels, "평균기온")

# X_train,X_test, y_train, y_test = train_test_data(X,y)
X_test = pd.DataFrame()

base_m = modeling(avg_tmp, X, y)

for k,v in label_dic.items() :
    
    X1, y1 = select_label(data, labels, k)
    
    # X_train1,X_test1, y_train1, y_test1 = train_test_data(X1,y1)
    
    print(v)
    
    m = modeling(v, X1, y1)
    
    X_test[k] = m.predict(X_test1)
    

# y_pred = base_m.predict(X_test)

# mae = mean_absolute_error(y_test, pred)

# print(mae)

In [10]:
def base_select_label(data, labels, label):
    
    X = data[["years", "months", "days"]]
    y = data[label]
    
    return X, y 

def train_test_data(X,y):
    
    X_train, X_test = X[:-365], X[-365:]
    y_train, y_test = y[:-365], y[-365:]
    
    return X_train, X_test, y_train, y_test

def progress(data, labels):
    
    
    data = data
    
    labels = labels
    
    for label in labels:
    
        X, y = select_label(data, labels, label)

        X_train,X_test, y_train, y_test = train_test_data(X,y)

        opt = make_model(X_train,y_train)

        label = label + "_model"
        
        print(label)

        globals()[label] = opt
    
def make_model(X_train, y_train):
    
    # 하이퍼파라미터 탐색 범위 정의
    param_space = {
        'learning_rate': (0.001, 1.00, 'uniform'),
        'max_depth': (3, 12),
        'n_estimators': (100, 2000),
        'subsample': (0.10, 1.00, 'uniform'),
        'colsample_bytree': (0.10, 1.00, 'uniform'),
        'gamma': (0, 10.0, 'uniform'),
    }

    # XGBoost 모델 초기화
    model = xgb.XGBRegressor(objective="reg:squarederror", n_jobs = -1, random_state=42)

    # BayesSearchCV를 사용하여 베이지안 최적화 수행
    opt = BayesSearchCV(
        model,
        param_space,
        scoring='neg_mean_absolute_error',
        n_iter=50,  # 반복 횟수
        cv=5,  # 교차 검증 폴드 수
        # n_jobs=1,  # 병렬 처리를 위한 CPU 코어 수 (-1은 모든 가능한 코어 사용)
    )
    
    # 최적의 하이퍼파라미터로 모델 학습
    opt.fit(X_train, y_train)

    # 최적의 하이퍼파라미터 출력|
    print("Best Parameters:", opt.best_params_)

    # 최적의 모델 평가 (여기서는 교차 검증 점수 출력)
    print("Cross-Validation Score:", -opt.best_score_)
    
    return opt
    

In [9]:
data.columns

Index(['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율',
       '평균기온', 'months', 'ds', 'days', 'years'],
      dtype='object')

In [11]:
labels = ["최고기온", "최저기온", "평균습도", "평균풍속","일사합", "일조율", "강수량"]

In [13]:
def train_test_data(X,y, level):
    
    X_train, X_test = X[:-level], X[-level:]
    y_train, y_test = y[:-level], y[-level:]
    
    return X_train, X_test, y_train, y_test

In [ ]:
for label in labels:

    X, y = base_select_label(data, labels, label)

    X_train,X_test, y_train, y_test = train_test_data(X,y, 365)

    opt = make_model(X_train,y_train)
    
    globals()[label+"_model"] = opt

Best Parameters: OrderedDict([('colsample_bytree', 0.6335108591777195), ('gamma', 5), ('learning_rate', 0.30522645243482066), ('max_depth', 6), ('n_estimators', 561), ('subsample', 0.7316463555463202)])
Cross-Validation Score: 3.197269372314757
Best Parameters: OrderedDict([('colsample_bytree', 1.0), ('gamma', 10), ('learning_rate', 0.2313810521218389), ('max_depth', 3), ('n_estimators', 100), ('subsample', 0.4491801521025067)])
Cross-Validation Score: 2.587316605715238
Best Parameters: OrderedDict([('colsample_bytree', 0.23675143044507335), ('gamma', 8), ('learning_rate', 0.03063065294495793), ('max_depth', 3), ('n_estimators', 617), ('subsample', 0.3275606379914264)])
Cross-Validation Score: 10.267357223331256
Best Parameters: OrderedDict([('colsample_bytree', 0.1922405112470672), ('gamma', 10), ('learning_rate', 0.8894590352877205), ('max_depth', 12), ('n_estimators', 100), ('subsample', 1.0)])
Cross-Validation Score: 0.7031825069354494
Best Parameters: OrderedDict([('colsample_bytr

In [134]:
make_model(X_train, y_train)

In [142]:
최고기온_model.best_params_

OrderedDict([('colsample_bytree', 1.0),
             ('gamma', 6),
             ('learning_rate', 0.13683314435428243),
             ('max_depth', 3),
             ('n_estimators', 100),
             ('subsample', 0.41477977896081264)])

In [144]:
365*3

1095

In [15]:
X = data[["years", "months", "days","최고기온", "최저기온", "평균습도", "평균풍속","일사합", "일조율", "강수량"]]
y = data["평균기온"]

In [16]:
X_train,X_test, y_train, y_test = train_test_data(X,y, 365)

In [17]:
main_model = make_model(X_train,y_train)

Best Parameters: OrderedDict([('colsample_bytree', 1.0), ('gamma', 3), ('learning_rate', 0.12492758147908661), ('max_depth', 12), ('n_estimators', 1326), ('subsample', 0.8014534674765247)])
Cross-Validation Score: 0.37252389005158915


In [18]:
X_test['최고기온'] = 최고기온_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['최저기온'] = 최저기온_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['평균습도'] = 평균습도_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['평균풍속'] = 평균풍속_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['일사합'] = 일사합_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['일조율'] = 일조율_model.best_estimator_.predict(X_test[["years", "months", "days"]])
X_test['강수량'] = 강수량_model.best_estimator_.predict(X_test[["years", "months", "days"]])

In [19]:
y_pred = main_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)

print(mae)

2.6142990210754413


In [20]:
forms = summit_form.copy()
forms.일시 = pd.to_datetime(forms.일시)
forms['days'] = forms.일시.dt.day
forms['months'] = forms.일시.dt.month
forms['years']= forms.일시.dt.year
forms.drop(['일시','평균기온'], axis = 1, inplace = True)

In [22]:
forms['최고기온'] = 최고기온_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['최저기온'] = 최저기온_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['평균습도'] = 평균습도_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['평균풍속'] = 평균풍속_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['일사합'] = 일사합_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['일조율'] = 일조율_model.best_estimator_.predict(forms[["years", "months", "days"]])
forms['강수량'] = 강수량_model.best_estimator_.predict(forms[["years", "months", "days"]])

In [155]:
import joblib
joblib.dump(최고기온_model, "./models/최고기온_1.joblib")
joblib.dump(최저기온_model, "./models/최저기온_1.joblib")
joblib.dump(평균습도_model, "./models/평균습도_1.joblib")

['./models/평균습도_1.joblib']

In [156]:
joblib.dump(mm, "./models/통합_1.joblib")

['./models/통합_1.joblib']

In [23]:
answer = main_model.predict(forms)

summit_form["평균기온"] = answer


# mae = mean_absolute_error(y_test, y_pred)

# print(mae)

In [24]:
summit_form.to_csv(f"./answer/xgb_7vars_{mae}.csv", index = False)